# Three-Body Bound State Calculation for ³H (Tritium)

This notebook computes the bound state of ³H (tritium) using the Faddeev method.
³H consists of one proton and two neutrons with quantum numbers J^π = 1/2^+, making it the simplest three-nucleon system.

## Installation Instructions

Before running this notebook, you need to compile the nuclear potential libraries and install Julia dependencies:

1. **Build the AV18 potential library:**
   ```bash
   cd NNpot
   make
   ```

2. **Install Julia packages:**
   ```bash
   cd swift
   julia setup.jl
   ```

## Initialize channel configuration and computational mesh for ³H bound state calculation

For ³H (tritium):
- One proton (p) + two neutrons (n,n)
- Total charge = +1, so MT = -0.5 
- Spin-parity quantum numbers: $J^π$ = $1/2^+$
- Total isospin T = 1/2
- No Coulomb repulsion between neutrons

In [1]:
include("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh

fermion=true; Jtot = 0.5; T = 0.5; Parity=1
lmax=4; lmin=0; λmax=20; λmin=0; s1=0.5; s2=0.5; s3=0.5; t1=0.5; t2=0.5; t3=0.5; MT=-0.5 # -0.5 for 3H
j2bmax=1.0  # Maximum J12 (two-body angular momentum)
nθ=12; nx=30; ny=30; xmax=20; ymax=20; alpha=1

α= α3b(fermion,Jtot,T,Parity,lmax,lmin,λmax,λmin,s1,s2,s3,t1,t2,t3,MT,j2bmax)  # the last variable define the parity of the pair system, if not defined then consider both parities. 
grid= initialmesh(nθ,nx,ny,Float64(xmax),Float64(ymax),Float64(alpha));

nothing

For J=0.5 T=0.5 parity=1 parity_pair=nothing Number of channels: 10
---The coupling coefficients are
 α3b| α2b |( l ( s1 s2 ) s12 ) J12 ( λ s3 ) J3 ,   J; ( t1 t2 ) T12 , t3 , T >
  1 |   1 |( 0 (0.5 0.5) 0.0)  0.0 ( 0 0.5) 0.5, 0.5; (0.5 0.5) 1.0, 0.5, 0.5 > 
  2 |   2 |( 0 (0.5 0.5) 1.0)  1.0 ( 0 0.5) 0.5, 0.5; (0.5 0.5) 0.0, 0.5, 0.5 > 
  3 |   2 |( 0 (0.5 0.5) 1.0)  1.0 ( 2 0.5) 1.5, 0.5; (0.5 0.5) 0.0, 0.5, 0.5 > 
  4 |   3 |( 1 (0.5 0.5) 0.0)  1.0 ( 1 0.5) 0.5, 0.5; (0.5 0.5) 0.0, 0.5, 0.5 > 
  5 |   3 |( 1 (0.5 0.5) 0.0)  1.0 ( 1 0.5) 1.5, 0.5; (0.5 0.5) 0.0, 0.5, 0.5 > 
  6 |   4 |( 1 (0.5 0.5) 1.0)  0.0 ( 1 0.5) 0.5, 0.5; (0.5 0.5) 1.0, 0.5, 0.5 > 
  7 |   5 |( 1 (0.5 0.5) 1.0)  1.0 ( 1 0.5) 0.5, 0.5; (0.5 0.5) 1.0, 0.5, 0.5 > 
  8 |   5 |( 1 (0.5 0.5) 1.0)  1.0 ( 1 0.5) 1.5, 0.5; (0.5 0.5) 1.0, 0.5, 0.5 > 
  9 |   6 |( 2 (0.5 0.5) 1.0)  1.0 ( 0 0.5) 0.5, 0.5; (0.5 0.5) 0.0, 0.5, 0.5 > 
 10 |   6 |( 2 (0.5 0.5) 1.0)  1.0 ( 2 0.5) 1.5, 0.5; (0.5 0.5) 0.0, 0.5, 0.5 > 
scaling fa

## Calculate deuteron bound state using nuclear potential

First we need the two-body (deuteron) binding energy as a reference threshold.
The deuteron is the bound state of a proton and neutron and provides the breakup threshold for ³H.

In [2]:
include("twobody.jl")
using .twobodybound

potname="AV18"
e2b, ψ =bound2b(grid, potname); 

nothing


Two-body channel configuration:
  Total angular momentum J = 1.0
  Parity = +
  Number of channels: 2
    Channel 1: 3S₁ (l=0, s=1.0)
    Channel 2: 3D₁ (l=2, s=1.0)

           TWO-BODY BOUND STATE ANALYSIS

Bound State #1:
  Binding Energy: -2.223939 MeV
  Total J^π = 1⁺

  Channel Composition:
    Channel 1: 3S₁ (l=0, s=1.0) - 94.24%
    Channel 2: 3D₁ (l=2, s=1.0) - 5.76%

  D-state Probability: 5.762%
  S-state Probability: 94.238%

SUMMARY: Found 1 bound state(s)
Binding energies (MeV): [-2.223939]


## Compute ³H bound state using Faddeev Equations

Now we solve the three-body Faddeev equations to find the ³H ground state.
The calculation includes:
- Nuclear attraction between all nucleon pairs (p-n, n-n interactions)
- Proper treatment of identical particle (neutron) symmetry
- Multi-channel coupling between different angular momentum states

In [ ]:
include("threebodybound.jl")
using .threebodybound

bound_states = ThreeBody_Bound(α, grid, potname, e2b);

nothing

## Alternative Solution: Malfiet-Tjon Method

Now we solve the same ³H bound state problem using the Malfiet-Tjon iterative method.
This approach reformulates the Faddeev equation as an eigenvalue problem where λ(E) = 1 
corresponds to the ground state energy, and uses the secant method for convergence.

In [3]:
include("MalflietTjon.jl")
using .MalflietTjon

# Solve using Malfiet-Tjon method with initial energy guesses
println("="^70)
println("    MALFIET-TJON METHOD FOR ³H GROUND STATE")
println("="^70)

# Use better initial energy guesses based on direct diagonalization result

    E0_guess = -7.5
    E1_guess = -6.5

result,ψtot, ψ3  = malfiet_tjon_solve(α, grid, potname, e2b,
                           E0=E0_guess,       # First energy guess (MeV)
                           E1=E1_guess,       # Second energy guess (MeV) 
                           tolerance=1e-6,     # Convergence tolerance |λ-1|
                           max_iterations=30,  # Reduced max iterations
                           verbose=true)       # Print iteration details

# Print comprehensive summary
print_convergence_summary(result)

nothing

    MALFIET-TJON METHOD FOR ³H GROUND STATE

         MALFIET-TJON EIGENVALUE SOLVER
Potential: AV18
Two-body threshold: -2.223939 MeV
Initial energy guesses: E0 = -7.5 MeV, E1 = -6.5 MeV
Convergence tolerance: 1.0e-6
Maximum iterations: 30
Eigenvalue method: Arnoldi (Krylov dim: 50)
Arnoldi tolerance: 1.0e-6
----------------------------------------------------------------------
  Building matrices...   8.623496 seconds (502.47 k allocations: 26.697 GiB, 6.50% gc time, 0.92% compilation time)

         REARRANGEMENT MATRIX SYMMETRY CHECK
Matrix dimensions: (9000, 9000)
Total elements: 81000000
Elements satisfying Rxy_31 = Rxy_32:  46980742 (58.0%)
Elements satisfying Rxy_31 = -Rxy_32: 34019258 (42.0%)
Elements satisfying neither:         0 (0.0%)
Maximum difference: 9.995869e-13
Tolerance: 1.000000e-12

✓ PASSED: All elements satisfy either Rxy_31 = Rxy_32 or Rxy_31 = -Rxy_32
  Mixed symmetric/antisymmetric relations found
  Computing RHS = LHS \ VRxy...   3.605499 seconds (19 allocati

## Wave Function Structure Analysis

Examine the spatial structure and channel composition of the ³H wave function.

## Summary

This calculation demonstrates the Faddeev method applied to ³H (tritium):

1. **Three-nucleon physics**: Exact treatment of correlations in the simplest neutron-rich nucleus
2. **Identical particle effects**: Proper symmetry treatment for two neutrons 
3. **Channel coupling**: Multi-channel approach captures nuclear structure complexity
4. **Neutron pairing**: Wave function analysis reveals neutron-neutron correlation patterns
5. **Benchmark system**: ³H serves as a testing ground for three-body methods in nuclear physics

The framework provides fundamental insights into few-nucleon systems and validates computational approaches for more complex nuclear calculations.